In [1]:
# pip install python-kasa flask requests picamera

In [2]:
'''
sudo iptables -t nat -A PREROUTING -p tcp --dport 80 -j REDIRECT --to-ports 8080 
sudo iptables -t nat -D PREROUTING -p tcp --dport 80 -j REDIRECT --to-ports 8080
''' 

import json # dhcp-lease-list
sec = {'kitchen_bulb_ip':"192.168.x.x", 
       'bedroom_bulb_ip':"192.168.x.x",
       'bedroom_ac_ip'  :"192.168.x.x", 
       'hub_ip'         :"192.168.x.x"} 

#with open('secret.json','w') as fw: json.dump(sec, fw)

In [3]:
import sqlite3, datetime, time, queue, threading, sys, os, asyncio, kasa

with open('secret.json') as fr: sec = json.load(fr) 

def initialize_database(): 
    con = sqlite3.connect(f'SHT40.{datetime.datetime.fromtimestamp(time.time()).strftime("%Y-%m-%dT%H:%M:%S")}.db')  
    cur = con.cursor() 
    cur.execute('CREATE TABLE IF NOT EXISTS log  (Temperature REAL, Humidity REAL, Photoresistor INTEGER, timestamp REAL);')
    con.commit() 
    con.close()
    
#initialize_database()

In [4]:
def database(): 
    con = sqlite3.connect('SHT40.2021-09-21T12:28:40.db')  
    cur = con.cursor() 
    while True: 
        opt, foo = q_in.get() # q.get() = opt_foo
        bar = False           # opt is a placeholder 
        try:
            if opt == 'insert':
                cur.execute('INSERT INTO log VALUES (?, ?, ?, ?);', 
                            (foo['Temperature (C)'], foo['Humidity (% rH)'], foo['Photoresistor'], time.time(),))  
            elif opt == 'sql':
                cur.execute(foo['sql'])  
                bar = cur.fetchall()
            elif opt == 'commit': 
                con.commit() 
                bar = 'Database commited'
                print('Database commited')
            elif opt == 'close': 
                con.commit()
                con.close()
                bar = 'Database commited and closed'
                print('Database commited and closed')
        except Exception as e:
            print('sql_exception', e, opt, foo)
            pass
        q_out.put(bar) 
        q_out.join()
        q_in.task_done()

q_in, q_out = queue.Queue(), queue.Queue() 
threading.Thread(target=database, name='thread-1', daemon=True).start() 

def database_operator(opt_foo): 
    q_in.join()        
    q_in.put(opt_foo)  
    bar = q_out.get()  
    q_out.task_done()   
    return bar

In [5]:
import struct, socket 
from random import randint

class WEBREPL:
    def upgrade_http_to_websocket(self):
        frw = self.s.makefile("rwb", 0)
        frw.write(b"""GET / HTTP/1.1\r\nHost: echo.websocket.org\r\nConnection: Upgrade\r\nUpgrade: websocket\r\nSec-WebSocket-Key: foo\r\n\r\n""")  
        while True:
            l = frw.readline() 
            if l == b'\r\n': break  
            # Response = '''HTTP/1.1 101 Switching Protocols\r\nUpgrade: websocket\r\nConnection: Upgrade\r\nSec-WebSocket-Accept: BM0S0+ghftShuFVHQATK/DBiJq8=\r\n\r'''
    
        buf = b''
        while 1:
            c = self.s.recv(1)
            buf += c
            if c == b':': 
                self.s.recv(1)
                break 
    
    def send(self, payload): 
        l = len(payload)
        if l > 125: raise 
        key = [randint(0, 255),randint(0, 255),randint(0, 255),randint(0, 255)]
        masked_payload = [ord(c)^key[i%4] for i, c in enumerate(payload) ]
        frame = struct.pack(f">{6+l}B", 0b10000001, 0b10000000 | l, *key,  *masked_payload) 
        self.s.send(frame)
        
    def recv(self): 
        buf = b''
        while True:
            _ = self.s.recv(1) 
            assert _ == b'\x81' # Fin + reserve + OpCode
            l = ord(self.s.recv(1))
            buf += self.s.recv(l)    
            if buf[-4:]==b'>>> ':  
                return buf.decode()   
        
    def __init__(self, host='192.168.1.1', port=8266, password='123456'): 
        self.s = socket.socket(socket.AF_INET, socket.SOCK_STREAM) 
        self.s.connect((host, port))
        self.upgrade_http_to_websocket()
        self.send(password) 
        self.send('\r')
        print(self.recv())
        
    def close(self):
        self.s.send(struct.pack(f">6B", 0x88, 0x80, 0x00, 0x00, 0x00, 0x00))

In [6]:
import dateutil, requests
UTC = dateutil.tz.gettz('UTC') 
class HUB:
    def __init__(self, temp_ceil = 28.5, lumi_ceil = 650):
        self.ac              = kasa.SmartPlug(sec['bedroom_ac_ip'])   
        self.bulb            = kasa.SmartBulb(sec['bedroom_bulb_ip'])   
        self.params          = { 'temp_ceil':temp_ceil,
                                 'lumi_ceil':lumi_ceil, 
                                'night_mode':False,
                               'manual_mode':False,
                                'sunrise_dt':None,
                                'sunset_dt' :None,
                                  'relay_on':False,
                                'bulb_timer':False
                               }
        threading.Thread(target=self.update).start()  
        time.sleep(3) 
            
    def update(self):
        lat, lng = (40.7063311, -73.9971931)
        onehour = datetime.timedelta(hours=1) 
        while 1:
            print('[Daily Sunrise & Sunset Update]')
            date = datetime.datetime.now().strftime('%Y-%m-%d')
            r = requests.get(f'https://api.sunrise-sunset.org/json?lat={lat}&lng={lng}&date={date}&formatted=0')
            if r.json()['status'] != 'OK': raise
            self.params['sunrise_dt'] = datetime.datetime.fromisoformat(r.json()['results']['sunrise'])
            self.params['sunset_dt']  = datetime.datetime.fromisoformat(r.json()['results']['sunset']) 
            while 1: 
                if self.params['sunrise_dt'].day != datetime.datetime.now().day: 
                    print('A New Day Has Come')
                    break
                asyncio.run(self.ac.update())   
                asyncio.run(self.bulb.update())
                if datetime.datetime.now().astimezone(UTC) < self.params['sunrise_dt'] + onehour or \
                   datetime.datetime.now().astimezone(UTC) > self.params['sunset_dt'] - onehour: # Daylight
                    self.params['night_mode'] = True 
                else:                                                        # At night
                    self.params['night_mode'] = False
                time.sleep(60)  
                if self.params['bulb_timer']: self.params.update({'bulb_timer':False})

hub = HUB()

[Daily Sunrise & Sunset Update]


In [ ]:
index_html_string = ''' <!DOCTYPE html>
<html>
  <head>
    <meta charset="utf-8">
    <title> hub </title>
    <style>
      .inp {height: 100px; width: 100px}  
      
      #slider {
        -webkit-appearance: none;
        width: 100%;
        height: 25px;
        background: #d3d3d3;
        outline: none;
        opacity: 0.7;
        -webkit-transition: .2s;
        transition: opacity .2s;
      }
      
      #slider:hover {
        opacity: 1;
      } 
      
      #slider::-webkit-slider-thumb {
        -webkit-appearance: none;
        appearance: none;
        width: 100px;
        height: 100px;
        background: #04AA6D;
        cursor: pointer;
      }
      
      #slider::-moz-range-thumb {
        width: 100px;
        height: 100px;
        background: #04AA6D;
        cursor: pointer;
      } 
      
      td {text-align: center; }
    </style>
  </head>
  <body> 
    <form method="POST" action="/" enctype="multipart/form-data"> 
      <table style="width: 500px;" > 
        <tr>
          <td class="inp">Light Mode:</td> 
          <td style="position: relative;">
            <input type="radio" name="manual_mode" value="1" {{'checked' if hub.params['manual_mode'] else ''}} class="inp">
            <span style="position: absolute;">Manu</span> 
          </td> 
          <td style="position: relative;">
            <input type="radio" name="manual_mode" value='0' {{'' if hub.params['manual_mode'] else 'checked'}} class="inp"> 
            <span style="position: absolute; top: 0px; left: 0px; ">Auto</span> 
          </td> 
        </tr>  
        <tr>
          <td>Max Celsius Degress:</td>
          <td><input type="text" name="temp_ceil" value="{{hub.params['temp_ceil']}}" style="font-size: 40px;" class="inp" id="output"/></td>
          <td><input type="submit" value="Submit" class="inp"></td>
        </tr>  
        <tr >
          <td colspan="3"><input type="range" min="1" max="100" value="{{ 10*hub.params['temp_ceil']-200}}" style="height:100px; width:100%" class="inp" id="slider"></td>
          <script>
            var output = document.getElementById("output");
            var slider = document.getElementById("slider");  
            slider.oninput = function() {
              output.setAttribute("value", 20+slider.value/10);   
            }
          </script> 
        </tr>
        <tr> 
          <td> <button type="button" onclick="light()" class="inp">Light</button> </td>
          <td> <button type="button" onclick="hdmi()" class="inp">HDMI</button> </td>
          <td> <button type="button" onclick="kich_lig()" class="inp">Kich_Lig</button> </td>
          <script>
            function light() { 
              var xmlHttp = new XMLHttpRequest();
              xmlHttp.open( "GET", "/remote?pin=15", false ); 
              xmlHttp.send( null );
            }; 
            
            function hdmi() {
              var xmlHttp = new XMLHttpRequest();
              xmlHttp.open( "GET", "/remote?pin=0", false ); 
              xmlHttp.send( null );
            }; 
            
            function kich_lig() {
              var xmlHttp = new XMLHttpRequest();
              xmlHttp.open( "GET", "/remote?pin=2", false ); 
              xmlHttp.send( null );
            }; 
          </script> 
        </tr>  
        <tr> 
          <td> <button type="button" onclick="kich_plg()" class="inp">Kich_Plg</button> </td> 
          <td> <button type="button" class="inp">Placeholder</button> </td>
          <td> <button type="button" class="inp">Placeholder</button> </td>
          <script>
            function kich_plg() { 
              var xmlHttp = new XMLHttpRequest();
              xmlHttp.open( "GET", "/remote?pin=4", false ); 
              xmlHttp.send( null );
            };  
          </script> 
        </tr>  
        <tr >
          <td colspan="3"> <img src="{{sec.cam_1_url}}"></img> </td>  
        </tr>
        <tr >
          <td colspan="3"> <img src="{{sec.cam_2_url}}"></img> </td> 
        </tr>
        
        <tr >
          <td>EA:DB:84:98:31:F4 </td> <td>MicroPython-9831f4 </td> <td> Controller</td> 
        </tr>
        <tr >
          <td>EA:DB:84:9C:54:9E </td> <td>MicroPython-9c549e </td> <td>unknown </td> 
        </tr>
        <tr >
          <td>EA:DB:84:9B:90:48 </td> <td>MicroPython-9b9048 </td> <td>HDMI Relay </td> 
        </tr>
        		  
        		  
        		  
        
      </table>  
      <br/> 
    </form>  
  </body>
</html> 
'''

import flask, logging

log = logging.getLogger('werkzeug') 
#log.setLevel(logging.ERROR) 

app = flask.Flask(__name__)  

@app.route("/", methods=['GET', 'POST'])
def index(): 
    req = flask.request 
    if req.method == 'GET':
        return flask.render_template_string(index_html_string, hub=hub, sec=sec) 
    elif req.method == 'POST':  
        hub.params['temp_ceil'] = float(req.form['temp_ceil']) 
        hub.params['manual_mode'] = int(req.form['manual_mode']) 
        return flask.redirect("/", code=302)
    
#kitchen_bulb = kasa.SmartBulb(sec['kitchen_bulb_ip']) 
kitchen_light_flag = False
kitchen_plug = kasa.SmartPlug(sec['kitchen_plug_ip']) 
@app.route("/remote", methods=['GET'])
def remote(): 
    pin = flask.request.args.get('pin', -1)
    if   pin ==  '0' : 
        if 'relay_ip' in hub.params: 
            if hub.params['relay_on']: 
                requests.post(f"http://{hub.params['relay_ip']}:{hub.params['relay_port']}/params",json={'switch': 0}) 
                hub.params.update({'relay_on':False})
                d = 'Relay Off'
            else: 
                requests.post(f"http://{hub.params['relay_ip']}:{hub.params['relay_port']}/params",json={'switch': 1}) 
                hub.params.update({'relay_on':True}) 
                d = 'Relay On' 
        else:
            d = 'No Relay' 
            
    elif pin ==  '2' :  
        webrepl = WEBREPL(host=sec['kitchen_light_ip'], password='123456') 
        global kitchen_light_flag 
        if kitchen_light_flag:  
            kitchen_light_flag = False
            code = '''
                    import time  
                    from machine import Pin, PWM 
                    PIN_MOSFET = 4
                    PIN_Servo = 5  
                    p0 = Pin(PIN_MOSFET, Pin.OUT, value=0) 
                    p0.on()       
                    pwm0 = PWM(Pin(PIN_Servo))    
                    pwm0.freq(50) 
                    i = 72 # off 
                    pwm0.duty(i)          
                    time.sleep(0.5)
                    i = 85 # idle
                    pwm0.duty(i)          
                    time.sleep(0.5) 
                    pwm0.deinit() 
                    p0.off()''' 
            d = 'KL_off'
        else:
            kitchen_light_flag = True
            code = '''
                    import time  
                    from machine import Pin, PWM 
                    PIN_MOSFET = 4
                    PIN_Servo = 5  
                    p0 = Pin(PIN_MOSFET, Pin.OUT, value=0) 
                    p0.on()       
                    pwm0 = PWM(Pin(PIN_Servo))    
                    pwm0.freq(50) 
                    i = 93 # on 
                    pwm0.duty(i)          
                    time.sleep(0.5)
                    i = 85 # idle
                    pwm0.duty(i)          
                    time.sleep(0.5)
                    p0.off()''' 
             
            d = 'KL_on'
        for cmd in code.splitlines():
            webrepl.send(cmd.strip())
            webrepl.send('\r')
            webrepl.recv()  
        webrepl.close()  
        
    elif pin ==  '222' :   
        global kitchen_bulb
        asyncio.run(kitchen_bulb.update()) 
        if kitchen_bulb.is_on:
            asyncio.run(kitchen_bulb.turn_off()) 
            asyncio.run(kitchen_bulb.update())
        else:
            asyncio.run(kitchen_bulb.turn_on()) 
            asyncio.run(kitchen_bulb.update())  
        d = 'Pin 2'
    elif pin ==  '4' :   
        global kitchen_plug
        asyncio.run(kitchen_plug.update()) 
        if kitchen_plug.is_on:
            asyncio.run(kitchen_plug.turn_off()) 
            asyncio.run(kitchen_plug.update())
        else:
            asyncio.run(kitchen_plug.turn_on()) 
            asyncio.run(kitchen_plug.update()) 
        d = 'Pin 4'
    elif pin ==  '5' :  
        d = 'Pin 5'
    elif pin == '12' :  
        hub.params['temp_ceil'] += 1
        d = f'Ceil: {hub.params["temp_ceil"]}'
    elif pin == '13' :  
        if hub.ac.is_on:
            asyncio.run(hub.ac.turn_off()) 
            asyncio.run(hub.ac.update())   
            d = 'A/C Off' 
        else:
            asyncio.run(hub.ac.turn_on()) 
            asyncio.run(hub.ac.update())  
            d = 'A/C On' 
    elif pin == '14' : 
        hub.params['temp_ceil'] -= 1
        d = f'Ceil: {hub.params["temp_ceil"]}'
    elif pin == '15' : # 
        if hub.bulb.is_on:
            asyncio.run(hub.bulb.turn_off()) 
            asyncio.run(hub.bulb.update())   
            hub.params['manual_mode'] = True
            d = 'LightOff'
        else:
            asyncio.run(hub.bulb.turn_on()) 
            asyncio.run(hub.bulb.update())   
            hub.params['manual_mode'] = False
            d = 'LightON'
    return flask.jsonify({'display':d})
    
@app.route("/log", methods=['POST'])
def log():
    T = flask.request.json['Temperature (C)']
    P = flask.request.json['Photoresistor']
    
    if T > hub.params['temp_ceil'] and hub.ac.is_on == False: 
        asyncio.run(hub.ac.turn_on()) 
        asyncio.run(hub.ac.update())  
    elif T < hub.params['temp_ceil'] - 1 and hub.ac.is_on == True:
        asyncio.run(hub.ac.turn_off()) 
        asyncio.run(hub.ac.update())  
        
    if hub.params['manual_mode'] or hub.params['bulb_timer']:
        pass
    else:
        hub.params.update({'bulb_timer':True})
        if hub.params['night_mode']: 
            if P < hub.params['lumi_ceil']:
                asyncio.run(hub.bulb.turn_on()) 
                asyncio.run(hub.bulb.update()) 
            elif hub.bulb.is_on: 
                asyncio.run(hub.bulb.turn_off()) 
                asyncio.run(hub.bulb.update())
        else:
            asyncio.run(hub.bulb.turn_off()) 
            asyncio.run(hub.bulb.update())
        
    database_operator(('insert', flask.request.json)) 
    return '' 

@app.route("/sql", methods=['POST'])
def sql():  
    return {'result':database_operator(('sql', flask.request.json)).__repr__() }
 
@app.route("/params", methods=['GET', 'POST'])
def params():   
    req = flask.request 
    if req.method == 'GET':
        return hub.params
    elif req.method == 'POST':  #  
        hub.params.update(req.json) 
        return {'return_code':100}

@app.route('/heartbeat', methods=['GET']) 
def heartbeat():
    database_operator(('commit', None))  
    return 'Cheers!'
 
app.run(sec['hub_ip'],'8080')
# database_operator(('sql', {'sql':'select * from log'})) 

In [ ]:
database_operator(('commit', ''))

In [ ]:
hub.params

In [ ]:
#!jupyter nbconvert --to script hub.ipynb